In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.utils.data import Dataset

import torchvision.datasets as dset
import torchvision.transforms as T
import torchvision.models as models

import numpy as np
from skimage import io

import timeit
import shutil
import os

import matplotlib.pyplot as plt
%matplotlib inline

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples

NUM_TEST = 300000

classes = ('airplane', 'automobile', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
class Cifar10TestDataset(Dataset):
    """Cifar10 test dataset."""

    def __init__(self, size, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.size = size
        self.root_dir = root_dir
        self.transform = transform
        self.train = False

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, '{}.png'.format(idx+1))
        image = io.imread(img_name)
        if self.transform:
            image = self.transform(image)
        return image

In [4]:
test_preprocess = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

batch_size = 1320

cifar10_test = Cifar10TestDataset(NUM_TEST, '../data/test', test_preprocess)
loader_test = DataLoader(cifar10_test, batch_size=batch_size)

In [5]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # read in N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image
    

class Pass(nn.Module):
    def forward(self, x):
        return x

In [6]:
# Verify that CUDA is properly configured and you have a GPU available
torch.cuda.is_available()

True

In [7]:
gpu_dtype = torch.cuda.FloatTensor

## Load Model

In [8]:
def load_checkpoint(filename='checkpoint.pth.tar'):
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        return torch.load(filename)
    else:
        return None

In [9]:
def make_resnet34():
    model = models.resnet34().type(gpu_dtype)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1).type(gpu_dtype)
    model.maxpool = Pass().type(gpu_dtype)
    model.avgpool = nn.AvgPool2d(kernel_size=4, stride=1, padding=0, ceil_mode=False, count_include_pad=True).type(gpu_dtype)
    model.fc = nn.Linear(512, 10).type(gpu_dtype)
    return model

def make_resnet101():
    model = models.resnet101().type(gpu_dtype)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1).type(gpu_dtype)
    model.maxpool = Pass().type(gpu_dtype)
    model.avgpool = nn.AvgPool2d(kernel_size=4, stride=1, padding=0, ceil_mode=False, count_include_pad=True).type(gpu_dtype)
    model.fc = nn.Linear(2048, 10).type(gpu_dtype)
    return model

In [10]:
checkpoint = load_checkpoint('resnet101-0.954.pth.tar')
final_model = make_resnet101()
final_model.load_state_dict(checkpoint['state_dict'])
print('Model loaded.')

=> loading checkpoint 'resnet101-0.954.pth.tar'
Model loaded.


In [ ]:
def get_preds(model, loader):
    pred_classes = []
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for x in loader:
        x_var = Variable(x.type(gpu_dtype), volatile=True)

        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        for pred in preds:
            pred_classes.append(classes[pred])
    return pred_classes

In [ ]:
pred_classes = get_preds(final_model, loader_test)

In [ ]:
f = open('../prediction.csv', 'w')
f.write('id,label\n')
for index, label in enumerate(pred_classes):
    f.write('{},{}\n'.format(index+1, label))

In [ ]:
f.close()